In [ ]:
# (그래프에 한글이 깨질경우) 아래 코드 실행 > '런타임: 세션 다시시작' 실행
!pip install matplotlib -U
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## **라이브러리 다운로드 및 불러오기**

In [ ]:
# 라이브러리 불러오기: 데이터 핸들링
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm, trange

# 라이브러리 불러오기: 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False  # plot "-" 부호 표기 설정
import plotly.express as px

import plotly.express as px
from plotly import graph_objects as go

In [ ]:
# 라이브러리 불러오기: 경고 안내 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

## **데이터**
  - 데이터명: 전자상거래 내, 카테고리별 고객 행동 데이터
  - 출처: 캐글 오픈 데이터(https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store)

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01.csv')
print(df.shape)
df.head()

(67501979, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [ ]:
# 데이터 변수별 의미
# event_time: 19년 11월 이벤트 발생 일시(협정 세계시(UTC))
# event_type: 이벤트 종류(view: 고객이 제품을 구경 / car: 고객이 장바구니에 담음 / purchase: 고객이 제품을 구매)
# product_id: 제품ID
# category_id: 제품 카테고리ID
# category_code: 제품의 카테고리 코드명(일반적으로 의미있는 카테고리에 존재하고 다양한 종류의 액세서리에는 건너뜀)
# brand: 상표명(널값 존재)
# price: 제품 가격
# user_id: 고객ID
# user_session: 고객세션

## **데이터 전처리**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67501979 entries, 0 to 67501978
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.5+ GB


In [ ]:
df.describe(include='all')

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
count,67501979,67501979,6.750198e+07,6.750198e+07,45603808,58277901,6.750198e+07,6.750198e+07,67501969
unique,2549559,3,NaN,NaN,129,4200,NaN,NaN,13776050
top,2019-11-15 02:51:51 UTC,view,NaN,NaN,electronics.smartphone,samsung,NaN,NaN,d99d91bf-40f8-4e29-9593-54b4a1826542
freq,243,63556110,NaN,NaN,16375000,7889245,NaN,NaN,4128
mean,NaN,NaN,1.251406e+07,2.057898e+18,NaN,NaN,2.924593e+02,5.386397e+08,NaN
std,NaN,NaN,1.725741e+07,2.012549e+16,NaN,NaN,3.556745e+02,2.288516e+07,NaN
min,NaN,NaN,1.000365e+06,2.053014e+18,NaN,NaN,0.000000e+00,1.030022e+07,NaN
25%,NaN,NaN,1.305977e+06,2.053014e+18,NaN,NaN,6.924000e+01,5.164762e+08,NaN
50%,NaN,NaN,5.100568e+06,2.053014e+18,NaN,NaN,1.657700e+02,5.350573e+08,NaN
75%,NaN,NaN,1.730075e+07,2.053014e+18,NaN,NaN,3.603400e+02,5.610794e+08,NaN


In [ ]:
# 데이터 결측치 확인
df.isnull().sum()

,0
event_time,0
event_type,0
product_id,0
category_id,0
category_code,21898171
brand,9224078
price,0
user_id,0
user_session,10


In [ ]:
# 결측치 제거(단, user_session만 제거)
df.dropna(subset=['user_session'], inplace=True)

In [ ]:
# 데이터프레임 정제
df = df[['event_time', 'event_type', 'product_id', 'user_id', 'user_session']]    # 변수 순서 변경
df.columns = ['일시(UTC)', '이벤트 타입', '상품ID', '고객ID', '고객 세션']        # 변수명 변경

In [ ]:
df.head()

,일시(UTC),이벤트 타입,상품ID,고객ID,고객 세션
0,2019-11-01 00:00:00 UTC,view,1003461,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [ ]:
# 함수 정의: UTC → KST
def convert_kst(utc_string):
  utc_string = utc_string.split(' UTC')[0]
  utc_string = datetime.strptime(utc_string,'%Y-%m-%d %H:%M:%S')   # Datetime 형태 변환
  df_kst = utc_string + timedelta(hours=9)                         # 협정 세계시간에 9시간 추가
  kst_datetime = df_kst.strftime('%Y-%m-%d %H:%M:%S')              # 일시 문자열 변환
  return kst_datetime


# 리스트 정의
kst_list = []
month_list = []
day_list = []


# 변수 생성: kst(전체일시/년/월/일)
for i in tqdm(df['일시(UTC)']):
  kst_time01 = convert_kst(i)
  kst_list.append(kst_time01)         # kst시간(전체 일시)

  kst_time02 = kst_time01.split(' ')[0].split("-")
  month_list.append(kst_time02[1])    # kst시간(월)
  day_list.append(kst_time02[2])      # kst시간(일)


# 데이터프레임 내, 데이터 생성
df['일시(KST)'] = kst_list
df['월(KST)'] = month_list
df['일(KST)'] = day_list

# 불필요한 데이터 요소 제거
df = df[df['월(KST)']=='11']

100%|██████████| 67501969/67501969 [18:49<00:00, 59762.34it/s]


In [ ]:
print(df['일(KST)'].max(), df['일(KST)'].min())

30 01


In [ ]:
df.head()

,일시(UTC),이벤트 타입,상품ID,고객ID,고객 세션,일시(KST),월(KST),일(KST)
0,2019-11-01 00:00:00 UTC,view,1003461,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2019-11-01 09:00:00,11,01
1,2019-11-01 00:00:00 UTC,view,5000088,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2019-11-01 09:00:00,11,01
2,2019-11-01 00:00:01 UTC,view,17302664,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2019-11-01 09:00:01,11,01
3,2019-11-01 00:00:01 UTC,view,3601530,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2019-11-01 09:00:01,11,01
4,2019-11-01 00:00:01 UTC,view,1004775,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2019-11-01 09:00:01,11,01


In [ ]:
# 데이터 저장(1)
#df.to_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01_kst.csv')

---

## **DAU(Daily Active User) 도출**

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01_kst.csv')
del df["Unnamed: 0"]
print(df.shape)
df.head()

(66913913, 8)


,일시(UTC),이벤트 타입,상품ID,고객ID,고객 세션,일시(KST),월(KST),일(KST)
0,2019-11-01 00:00:00 UTC,view,1003461,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2019-11-01 09:00:00,11,1
1,2019-11-01 00:00:00 UTC,view,5000088,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2019-11-01 09:00:00,11,1
2,2019-11-01 00:00:01 UTC,view,17302664,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2019-11-01 09:00:01,11,1
3,2019-11-01 00:00:01 UTC,view,3601530,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2019-11-01 09:00:01,11,1
4,2019-11-01 00:00:01 UTC,view,1004775,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2019-11-01 09:00:01,11,1


In [ ]:
# 19년 11월 일일 활성 고객 수 도출
day_df = df[['일(KST)', "고객ID"]]                 # 필요한 변수만 선별
result = day_df.drop_duplicates()                  # 데이터프레임 내 변수별 중복된 데이터 제거
grouped = result.groupby(['일(KST)']).size()       # 일자별 방문 고객

print('19년 11월 평균 방문 고객 수: ', round(grouped.mean()))
grouped

19년 11월 평균 방문 고객 수:  285986


,0
일(KST),
1,163146
2,229279
3,244636
4,262783
5,270277
6,256212
7,282676
8,281517
9,271257


In [ ]:
# 요일 데이터 생성

# 19년 11월 전체일자 생성
Nov_day = pd.date_range('2019-11-01', '2019-11-30', freq='D').to_frame().rename(columns={0:'날짜'}).reset_index(drop=True)

# 전체일자를 Date형태로 변환
day_list = []
for i in Nov_day["날짜"]:
  day_list.append(i.strftime("%Y-%m-%d"))
date_idx = pd.DatetimeIndex(day_list)

# Date형태로 변환된 전체일자를 요일로 변환
week_idx_list = []
for i in tqdm(date_index):
  day_of_week = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
  week_idx = i.weekday()
  week_idx_list.append(day_of_week[week_idx])

# 데이터프레임 생성
date_week_day = pd.DataFrame({'날짜':date_index, '요일':week_idx_list})
print(date_week_day.shape)
date_week_day

(30, 2)


,날짜,요일
0,2019-11-01,금요일
1,2019-11-02,토요일
2,2019-11-03,일요일
3,2019-11-04,월요일
4,2019-11-05,화요일
5,2019-11-06,수요일
6,2019-11-07,목요일
7,2019-11-08,금요일
8,2019-11-09,토요일
9,2019-11-10,일요일


In [ ]:
# 데이터 저장(2)
#date_week_day.to_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/date_week_day.csv')

In [ ]:
## 2019년 11월 일일 활성 유저 수(DAU)를 분석했다.

# 19년 11월 평균 285,986명이 전자상거래 시장을 방문했으며, 그중 월 중순인 '11, 12, 14, 15, 16, 17, 18, 19, 20, 21일'에 평균 보다 많은 고객들이 방문하는 것을 확인했다.
# 또한, 가장 많은 활성 유저가 방문한 일자는 '17일'로써 평균 보다 최대 1.7배(499,430/285,986) 많은 것을 확인했다.

# 이에, 가장 많은 고객 방문이 이뤄진 17일을 대상으로 고객 전환율을 분석하여 매출 극대화를 위한 착안점을 찾고자 한다.

---

## **홈페이지 내 고객 체류시간**

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01_kst.csv')
del df["Unnamed: 0"]
print(df.shape)
df.head()

(66913913, 8)


,일시(UTC),이벤트 타입,상품ID,고객ID,고객 세션,일시(KST),월(KST),일(KST)
0,2019-11-01 00:00:00 UTC,view,1003461,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2019-11-01 09:00:00,11,1
1,2019-11-01 00:00:00 UTC,view,5000088,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2019-11-01 09:00:00,11,1
2,2019-11-01 00:00:01 UTC,view,17302664,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2019-11-01 09:00:01,11,1
3,2019-11-01 00:00:01 UTC,view,3601530,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2019-11-01 09:00:01,11,1
4,2019-11-01 00:00:01 UTC,view,1004775,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2019-11-01 09:00:01,11,1


In [ ]:
# 데이터 타입 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66913913 entries, 0 to 66913912
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   일시(UTC)  object
 1   이벤트 타입   object
 2   상품ID     int64 
 3   고객ID     int64 
 4   고객 세션    object
 5   일시(KST)  object
 6   월(KST)   int64 
 7   일(KST)   int64 
dtypes: int64(4), object(4)
memory usage: 4.0+ GB


In [ ]:
# 고객 체류시간 측정을 위한 데이터 정제

# 19년 11월 가장 많은 고객이 체류한 17일 데이터 선별
day17_df = df[df['일(KST)'] == 17]

print(type(day17_df))
print(day17_df.shape)
day17_df.tail()

<class 'pandas.core.frame.DataFrame'>
(6697509, 8)


,일시(UTC),이벤트 타입,상품ID,고객ID,고객 세션,일시(KST),월(KST),일(KST)
43895180,2019-11-17 14:59:59 UTC,view,3900536,518451111,8020aa8d-9a25-47d8-bf28-da4d9b13ef16,2019-11-17 23:59:59,11,17
43895181,2019-11-17 14:59:59 UTC,view,5300128,514822215,35ca3007-4766-4137-b392-f33221addfeb,2019-11-17 23:59:59,11,17
43895182,2019-11-17 14:59:59 UTC,view,1201343,513540069,13697709-ceb9-48bc-a6da-624bd9b12186,2019-11-17 23:59:59,11,17
43895183,2019-11-17 14:59:59 UTC,purchase,42100000,549901620,659e2f4e-a4e1-442f-acb9-0d083628c9f8,2019-11-17 23:59:59,11,17
43895184,2019-11-17 14:59:59 UTC,cart,1005115,513331182,9602ff9c-79e2-4cd1-8082-6e1e412f3de6,2019-11-17 23:59:59,11,17


In [ ]:
day17_df['일시(KST)'] = pd.to_datetime(day17_df['일시(KST)'])

In [ ]:
day17_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6697509 entries, 37197676 to 43895184
Data columns (total 8 columns):
 #   Column   Dtype         
---  ------   -----         
 0   일시(UTC)  object        
 1   이벤트 타입   object        
 2   상품ID     int64         
 3   고객ID     int64         
 4   고객 세션    object        
 5   일시(KST)  datetime64[ns]
 6   월(KST)   int64         
 7   일(KST)   int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 459.9+ MB


In [ ]:
# 데이터 저장(3)
#day17_df.to_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01_kst_day17.csv')

___

In [ ]:
# 홈페이지 내 고객 체류시간 분석

# '고객 세션'을 기준으로 행동일자 및 시간의 최초(min) 및 최근(max) 결과 데이터 도출
Max_grouped = day17_df.groupby(['고객 세션', '고객ID'])["일시(KST)"].max()
Max_Time_DF = pd.DataFrame(Max_grouped)
Max_Time_DF.columns = ["최근행동"]

Min_grouped = day17_df.groupby(['고객 세션', '고객ID'])["일시(KST)"].min()
Min_Time_DF = pd.DataFrame(Min_grouped)
Min_Time_DF.columns = ["과거행동"]

# 데이터프레임 결합 및 변수 생성
NEW_DF = pd.concat([Max_Time_DF, Min_Time_DF], axis = 1)     # 두 데이터프레임 결합(열 단위)
NEW_DF["세션일시"] = NEW_DF["최근행동"] - NEW_DF["과거행동"] # 최근에서 최초 이벤트를 빼서 체류 결과 도출

# 고객별 체류 시간이 높은 고객부터 정렬
NEW_DF.sort_values(by=["세션일시"],
                   axis = 0,            # ascending = False 내림차순 정렬
                   ascending = False,   # axis = 0 행인덱스 기준 정렬
                   inplace = True)      # inplace = True 원본 대체 여부
NEW_DF

,,최근행동,과거행동,세션일시
고객 세션,고객ID,,,
a7c5906e-5dd8-4175-aeca-eb5615844e67,565022209,2019-11-17 22:54:37,2019-11-17 01:08:20,0 days 21:46:17
f2d487ec-1a93-47cd-916a-5e46db85472c,568848552,2019-11-17 23:09:19,2019-11-17 01:50:50,0 days 21:18:29
c28e0611-7590-49cb-9c19-1d4f6540592a,546179105,2019-11-17 22:30:13,2019-11-17 01:26:06,0 days 21:04:07
a7b1bde2-4493-4c4b-9517-8b8a0a9cf494,557268031,2019-11-17 22:42:46,2019-11-17 02:06:13,0 days 20:36:33
c5fc6a55-1735-4636-8490-b6dfd060164c,554760857,2019-11-17 22:13:34,2019-11-17 02:01:21,0 days 20:12:13
...,...,...,...,...
51352def-f9dc-43f7-91cf-c23d9383b7aa,544820104,2019-11-17 05:03:56,2019-11-17 05:03:56,0 days 00:00:00
a8adce34-24af-4224-90fc-b3813cf955df,518614294,2019-11-17 15:37:37,2019-11-17 15:37:37,0 days 00:00:00
d40ac5a9-1340-49df-a99f-f9a2bbd314f6,568616224,2019-11-17 21:57:49,2019-11-17 21:57:49,0 days 00:00:00


In [ ]:
## 가장 많은 고객이 방문한 17일의 체류시간을 분석했다.

# 사용자 세션은 고객의 행동 단위로써 11월 17일 가장 많은 시간을 체류한 고객은 '565022209'로써 21시간 46분 17초를 접속한 것을 확인했다.
# 해당 결과는 향후 접속시간에 따른 고객 등급화를 추진할 수 있을 것으로 예상된다.

## **퍼널 분석(Funnel Analysis)**

In [ ]:
# 데이터 불러오기
day17_df = pd.read_csv('/content/drive/MyDrive/4. Project/2. 홍생정 블로그/eCommerce_CBD_01_kst_day17.csv')
day17_df = day17_df[['일시(KST)', '이벤트 타입', '고객 세션']]
day17_df = day17_df.reset_index()

del day17_df['index']
print(day17_df.shape)
day17_df.head()

(6697509, 3)


,일시(KST),이벤트 타입,고객 세션
0,2019-11-17 00:00:00,view,20edad9e-c725-4cdf-9519-22b400233351
1,2019-11-17 00:00:00,view,2d5f2702-de13-48c6-86bc-e9975125461c
2,2019-11-17 00:00:00,view,5efd8289-8268-4138-af9f-fbf49451d55b
3,2019-11-17 00:00:00,view,fdddfbe0-bece-46aa-9c57-66efef984d3a
4,2019-11-17 00:00:00,purchase,1cd1ae0b-bb9a-4974-be8d-d3af291ad465


In [ ]:
# 데이터 타입 확인
day17_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6697509 entries, 0 to 6697508
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   일시(KST)  object
 1   이벤트 타입   object
 2   고객 세션    object
dtypes: object(3)
memory usage: 153.3+ MB


In [ ]:
# 퍼널 분석: 상품보기 → 장바구니담기 → 결제(1)

# 이벤트 접근여부를 최초 접속시간을 활용
grouped = day17_df.groupby(["고객 세션", "이벤트 타입"])["일시(KST)"].min()

# 퍼널 단계를 구분하고자 별도의 데이터프레임 생성
funnel_steps = pd.DataFrame({'단계(Steps)': [1, 2, 3]}, index = ['view', 'cart', 'purchase'])

# 고객 이벤트 타입별 일자 및 시간에 대한 퍼널결합
grouped = pd.DataFrame(grouped).merge(funnel_steps, left_on = '이벤트 타입', right_index = True)
grouped

,,일시(KST),단계(Steps)
고객 세션,이벤트 타입,,
00000616-f016-4c01-b323-438486d9d3ee,view,2019-11-17 14:35:46,1
00002e0a-8b74-4c72-bb5a-99e04fa264da,view,2019-11-17 17:22:06,1
00004c44-9765-4216-aafd-0ede55aad240,view,2019-11-17 23:00:14,1
00005114-2d52-4a59-ac08-ec71bbaf24db,view,2019-11-17 22:28:30,1
00006a96-5e74-427a-97f9-5d9cdf4a2de1,view,2019-11-17 21:59:41,1
...,...,...,...
fffe04cb-aa10-468d-a4eb-873c19377644,purchase,2019-11-17 20:28:00,3
ffffb69f-e3f9-4271-876f-d894920fa911,purchase,2019-11-17 23:47:28,3
ffffde87-ca37-4bef-a2fb-6901472fd8c8,purchase,2019-11-17 19:59:07,3


In [ ]:
# 퍼널 분석: 상품보기 → 장바구니담기 → 결제(2)

funel = grouped.reset_index().pivot(index="고객 세션", columns = '단계(Steps)', values = '일시(KST)')
funel.columns = funnel_steps.index
funel

,view,cart,purchase
고객 세션,,,
00000616-f016-4c01-b323-438486d9d3ee,2019-11-17 14:35:46,NaN,NaN
00002e0a-8b74-4c72-bb5a-99e04fa264da,2019-11-17 17:22:06,NaN,NaN
00004c44-9765-4216-aafd-0ede55aad240,2019-11-17 23:00:14,NaN,NaN
00005114-2d52-4a59-ac08-ec71bbaf24db,2019-11-17 22:28:30,2019-11-17 22:31:26,NaN
00006a96-5e74-427a-97f9-5d9cdf4a2de1,2019-11-17 21:59:41,NaN,NaN
...,...,...,...
ffffee94-7d46-454c-b12b-19e6f78fa18b,2019-11-17 17:12:53,NaN,NaN
fffff1dc-0841-4b56-9658-d5dcdc8be2d0,2019-11-17 18:32:17,NaN,NaN
fffff736-405e-485f-ac84-6a622ef44329,2019-11-17 14:38:10,2019-11-17 14:38:12,2019-11-17 14:40:56


In [ ]:
# 퍼널 분석: 상품보기 → 장바구니담기 → 결제(3)

step_values_list = []
for columns in funel.columns:
  step_values = funel[columns].notnull().sum()
  step_values_list.append(step_values)
  print('#11월', columns, '단계의 활성 고객 수:', end = ''); print(step_values, '명')

#step_values_list

#11월 view 단계의 활성 고객 수:1007174 명
#11월 cart 단계의 활성 고객 수:235220 명
#11월 purchase 단계의 활성 고객 수:119216 명


In [ ]:
# 퍼널 분석: 상품보기 → 장바구니담기 → 결제(4)

[step_values_list[i] for i in range(0, 3)]

fig = go.Figure(go.Funnel(
    y = ['view', 'cart', 'purchase'],
    x = step_values_list,
    textposition = "inside",              # 텍스트 위치
    textinfo = "value + percent initial",
    opacity = 1,
    marker = {"color": ["deepskyblue", "lightblue", "powderblue"], "line": {"width": [0, 0, 2], "color": ["wheat", "wheat", "red"]}}))

fig.update_layout(width=1000, height=500)
fig.show()

In [ ]:
# 19년 11월 전자상거래 시장에서 가장 많은 고객이 방문한 17일을 대상으로 퍼널 분석을 수행했다.
# 퍼널 분석을 통해 최초 유입부터 최종 목적지까지 어느 정도 전환이되었는지 확인하고자 했다.

# 고객이 상품을 보고 장바구니에 담기까지 23.4% 전환율을 보였고, 이후 최종 결제까지 50.7% 전환율을 보였다.
# 이를 쉽게 해석하면 상품을 보는 100명 중 23명이 장바구니에 담았고, 그중 12명이 상품 결제를 수행했다고 해석 가능하다

# 여기서, 흥미로운 것은 '장바구니 담기부터 결제 단계'보다 '상품 보기부터 장바구니 담기 단계'의 전환율이 비교적 낮은 것을 확인했다.
# 이를 해석하면 상품 보기에서 장바구니 담기의 전환율이 높아진다면 결제까지는 비교적 쉽게 수행될 수 있다고 추측 가능하다

___

In [ ]:
# 생성 데이터 설명
# eCommerce_CBD_01: 고객 행동 데이터(원본)
# eCommerce_CBD_01_kst: 한국 시간으로 변경된 고객 행동 데이터
# date_week_day: 19년 11월 일별 요일
# eCommerce_CBD_01_kst_day17: 한국시간 19년 17일 고객 행동 데이터

___

#### **향후 ML모델링 및 분류 분석을 연구하고 싶으면 아래 링크 참고**
- https://www.kaggle.com/code/tshephisho/ecommerce-behaviour-using-xgboost

___